In [ ]:
#try to add how many repetitive rounds...enter number of repetitions
round=1
repetitions = 10
while round < repetitions + 1:
    round += 1
    if __name__ == '__main__':
        env = gym.make('2048-v0')

        #model hyperparameters (commetn what doesn need to be rewritten)
        #ALPHA = 0.1
        GAMMA = 0.99
        #As already some knowledge, less randomness. still decreasing. reduce start             with each repetition
        EPSILON = 1/round
        #this variable counts how many times we have won.
        won = 0
        #value we want to reach due to memory restrictions.
        #GOAL = 512
        max_value = 0
        #num_won = []
        #max_values = []
        #avg_values = []
        max_total = 0

        #construct state space
    
        #states = []
        #Q = {}

        #numGames = 5000
        BATCH_SIZE = int(numGames/100)
        
        for i in range(numGames):
            game_won = False
            observation = numpy_transformer_light(env.reset())
            s = tuple(observation)
            #check if observation isn't already in states
            if observation not in states:
                states.append(observation)
                for a in range(4):
                    #we changed the state to a to avoid a TypeError,
                    #because lists aren't hashable.S
                    Q[tuple(s),a] = 0
            rand = np.random.random()

            done = False
            epRewards = 0
            while not done:      
                # choice and validity check to avoid dead ends (loops)
                a, observation_, reward, done = choseandcheck(Q, a, observation, EPSILON, env, s)

                s_ = tuple(observation_)
                if observation_ not in states:
                    states.append(observation_)
                    for a in range(4):
                        #we changed the state to a to avoid a TypeError,
                        #because lists aren't hashable.
                        Q[tuple(s_),a] = 0
                        

                rand = np.random.random()
                if rand < (1-EPSILON):
                    a_ = maxAction(Q,s_)
                else: 
                    a_ = np.random.randint(0,4)
                epRewards += reward
                Q[s,a] = Q[s,a] + ALPHA*(reward + GAMMA*Q[s_,a_] - Q[s,a])
                
                # no more needed. that does not check for validity of move. we incorporated thi in choseandcheck
                # s,a = s_,a_

                #checks if the GOAL is reached. Sets the done to True to avoid KeyError (a higher state can be reached
                # but we don't want to reach it, because we don't have in in our state_space.)
                if get_max(observation_)==GOAL:
                    game_won = True
                
                if get_max(observation_)>max_value:
                    max_value = get_max(observation_)
                    
                observation=observation_
            
            
            if game_won == True:
                won+= 1

            if EPSILON > 0.0002:
                EPSILON -= 2/(numGames)  
            else:
                EPSILON = 0.0002
            totalRewards.append(epRewards)
            max_total += get_max(observation)
            
            if (i+1)%(BATCH_SIZE) == 0:
                new = won/(BATCH_SIZE)
                num_won.append(new)
                max_values.append(max_value)
                average = max_total/BATCH_SIZE
                avg_values.append(average)
                won = 0
                max_value = 0
                max_total = 0
                print("Episode: " + str(i+1))
    


x = [j for j in range(1,101)]
    import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

plotLearning(x,num_won,'Batch','Won games in percentage', 'Win-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,num_won)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=num_won, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

plotLearning(x,max_values,'Batch','Max_values', 'Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,max_values)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=max_values, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


plotLearning(x,avg_values,'Batch','Avg_values', 'Average-Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,avg_values)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=avg_values, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

    plt.show()
            